In [2]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import dotenv
import os

In [3]:
load_dotenv()
LLAMA_3_API_KEY = os.getenv("LLAMA_3_API_KEY")

In [35]:
llm = ChatGroq(
    temperature=0,
    groq_api_key=LLAMA_3_API_KEY,
    model_name="llama-3.1-70b-versatile",
    seed=42  # Đặt seed để đảm bảo kết quả luôn giống nhau
)

f:\Năm 3 - HK1\DoAnCS311\venv\Lib\site-packages\langchain_groq\chat_models.py:362: UserWarning: WARNING! seed is not default parameter.
                    seed was transferred to model_kwargs.
                    Please confirm that seed is what you intended.
  warnings.warn(


In [84]:
from pydantic import BaseModel, Field, EmailStr, field_validator
from typing import List, Optional
from datetime import date

# ForgivingDate model with date validation
class ForgivingDate(BaseModel):
    day: int
    month: int
    year: int

    @field_validator('day')
    def day_validator(cls, v):
        if v == None:
            v = None
        if v > 31 or v < 0:
            raise ValueError("Day not in range")
        return v

    @field_validator('month')
    def month_validator(cls, v):
        if v == None:
            v = None
            
        if v > 12 or v < 0:
            raise ValueError("Month not in range")
    
        return v

# Contact model with email validation
class Contact(BaseModel):
    name: str
    phone_number: str
    email: Optional[str] = Field(None, description="Email address")
    linkedin: str
    location: str = Field(
        default_factory=str,
        description="Complete street address wherever possible."
    )

# DateRange model containing start and end dates
class DateRange(BaseModel):
    start: ForgivingDate
    end: ForgivingDate = Field(description="Date of the end", default=ForgivingDate(day=date.today().day, month=date.today().month, year=date.today().year))

# Skills model with various fields for skills information
class Skills(BaseModel):
    name: str = Field(description="Extract the technical tools in the following text. Technical tools are generally in 2-3 words")

# Experience model containing details about work experiences
class Experience(BaseModel):
    dates: DateRange
    title: str = Field(description="Title of the role")
    num_experience: float = Field(description='Years of experience deducted from the (number of days between the dates)/365')
    company: str = Field(description="The employer")
    skills: List[Skills]
    description: str = Field(description="Detailed description of the experience")

# Education model for education details
class Education(BaseModel):
    college: str = Field(description='Institution from which the person received their degree')
    dates: DateRange

# Project model to capture project details
class Project(BaseModel):
    dates: DateRange
    title: str = Field(description="Title of the role")
    num_experience: float = Field(description='Years of experience deducted from the (number of days between the dates)/365')
    name_project: str = Field(description="Name of the project")
    skills: List[Skills]
    description: str = Field(description="Detailed description of the project")

# class CertificateAward(BaseModel):
#     name: str = Field(description="The name of the certificate or award")
#     issuing_organization: str = Field(description="The organization that issued the certificate or award")


# Candidate model containing all information about the candidate
class Candidate(BaseModel):
    contact: Contact
    
    programming_language: List[str] = Field(description="The programming language")
    language: List[str] = Field(description="The spoken/written language")
    tool: List[str] = Field(description="Technical tool, generally in 2-3 words.")
    soft_skills: List[str] = Field(description="Name of the soft skills")
    
    # education: List[Education]
    experience: List[Experience]
    projects: List[Project]
    
    # certificates: List[CertificateAward]

In [85]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Candidate)

prompt = PromptTemplate(
    template="""Extract the following structured information from the provided CV text. If information is missing, leave it blank. 
    \n{format_instructions}\n{query}\n""",
     input_variables=["query"],
     partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

In [86]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(
     r'F:\Năm 3 - HK1\DoAnCS311\data\CVs\5.pdf'
)
pages = []
text = ""
for doc in loader.lazy_load():
     pages.append(doc)
     text += doc.page_content


invalid pdf header: b'\n%PDF'


In [87]:
pages

[Document(metadata={'source': 'F:\\Năm 3 - HK1\\DoAnCS311\\data\\CVs\\5.pdf', 'page': 0}, page_content='User 1\nAI ENGINEER\nTHÔNG TIN CÁ NHÂN\n\ue11b 0935023456\n✉ nguyenduytamanh@example2\n.com\n\uf3c5 Th ủ  Đ ứ c, H ồ  Chí Minh\nCÁC K Ỹ  N Ă NG\nK Ĩ  N Ă NG CHUYÊN MÔN\n•Thành th ạ o: HTML, CSS, \nJavaScript, ReactJS, Redux \nFramework: Next.js, Tailwind \nCSS, Bootstrap\n•Công c ụ : Git, Webpack, Figma\n•Testing: Jest, Cypress \nK Ĩ  N Ă NG M Ề M\n•Giao ti ế p và làm vi ệ c nhóm hi ệ u \nqu ả \n•Qu ả n lý th ờ i gian t ố t\n•K ỹ  n ă ng gi ả i quy ế t v ấ n đ ề  \nTHÔNG TIN THÊM\nM Ụ C TIÊU NGH Ề  NGHI Ệ P\n- Phát tri ể n và đ óng góp trong l ĩ nh v ự c AI, đ ặ c bi ệ t là Machine Learning và NLP. - - - \nH ư ớ ng đ ế n tr ở  thành AI Engineer chuyên nghi ệ p, t ố i ư u hóa các gi ả i pháp công ngh ệ  \nph ụ c v ụ  doanh nghi ệ p và xã h ộ i. \nKINH NGHI Ệ M LÀM VI Ệ C\nTh ự c t ậ p sinh Front-End Developer 0 6 / 2 0 2 4 - 0 9 / 2 0 2 4 \nCông ty XYZ \n•Xây d ự ng giao di ệ n ng ư ờ

In [88]:

chain.invoke({"query":text})

{'contact': {'name': 'Nguyen Duy Tamanh',
  'phone_number': '0935023456',
  'email': 'nguyenduytamanh@example2.com',
  'linkedin': '',
  'location': 'Thủ Đức, Hồ Chí Minh'},
 'programming_language': ['HTML', 'CSS', 'JavaScript'],
 'language': [],
 'tool': ['Git',
  'Webpack',
  'Figma',
  'Jest',
  'Cypress',
  'Next.js',
  'Tailwind CSS',
  'Bootstrap'],
 'soft_skills': ['Giao tiếp và làm việc nhóm hiệu quả',
  'Quản lý thời gian tốt',
  'Kỹ năng giải quyết vấn đề'],
 'experience': [{'dates': {'start': {'day': 6, 'month': 2, 'year': 2024},
    'end': {'day': 9, 'month': 2, 'year': 2024}},
   'title': 'Thực tập sinh Front-End Developer',
   'num_experience': 0.25,
   'company': 'Công ty XYZ',
   'skills': [{'name': 'ReactJS'}, {'name': 'Tailwind CSS'}],
   'description': 'Xây dựng giao diện người dùng cho hệ thống quản lý khách hàng (CRM) bằng ReactJS và Tailwind CSS. Tối ưu hiệu suất trang web, giảm thời gian tải xuống dưới 3 giây. Phối hợp với nhóm Backend để tích hợp API và đảm bảo 

In [92]:
import pandas as pd

# Gọi chuỗi chain.invoke và lưu kết quả
result = chain.invoke({"query": text})

# Chuyển kết quả thành một DataFrame (giả sử kết quả là một từ điển hoặc chuỗi)
if isinstance(result, dict):
    df = pd.DataFrame([result])  # Nếu là dict, chuyển đổi sang DataFrame
elif isinstance(result, str):
    df = pd.DataFrame([{"response": result}])  # Nếu là chuỗi, gói trong một cột 'response'
else:
    raise ValueError("Result format not supported")

# Lưu vào tệp CSV
output_file = "output.csv"
df.to_csv(output_file, index=False, encoding='utf-8')

print(f"Result saved to {output_file}")


Result saved to output.csv


In [91]:
df['experience']

0    [{'dates': {'start': {'day': 6, 'month': 2, 'y...
Name: experience, dtype: object